In [1]:
import sys
sys.path.append("/Users/pan/Projects/seqpy/")
from src import *
import matplotlib.pyplot as plt
%matplotlib widget

## Generating a single pulse

In [2]:
# default settings, no need to call if no configuration files exist
config.update("RELATIVE_TIMING", True)
config.update("SAMPLING_FREQUENCY", 2.4e9)

In [3]:
a = Gaussian(width=100, plateau=100).shift(100)
b = Drag(width=100).shift(-1000)
c = Cosine(width=200, plateau=200).shift(1000)
fig, ax = plt.subplots()
ax.plot(2 * (0.1 + 0.2 * a + b * 0.5 + c * 0.8 + 0.3).waveform)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Register pulses to sequence

In [4]:
x, y, z = sweepables(3)
a = 0.2 * Gaussian(width=100, plateau=100).shift(x)
b = 0.3 * Drag(width=100)
c = 0.4 * Cosine(width=200, plateau=200)
c1 = Carrier(phases=0, frequencies=2e7)
c2 = Carrier(phases=z, frequencies=3e8)
c3 = Carrier(phases=(0,0), frequencies=(1e7,3e7))
s = Sequence(n_channels=2)
s.register(0, a, c1, channel=0)
s.register(y, b, c2, channel=0)
s.register(2000, c, c3, channel=1)
s.trigger_pos = 0
s.subs(x, 100)
s.subs(y, 1000)
s.subs(z, 90)
s.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# TODO: the order of the symbols is distorted, could call subs with name to avoid
s.dump("test.json")
s.load("test.json")
s.subs("x0", 0.8)
s.subs("x1", 1000)
s.subs("x2", 90)
s.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Using absolute timing

In [9]:
config.update("RELATIVE_TIMING", False)
config.update("SAMPLING_FREQUENCY", 1e9)
a = 0.3 * Gaussian(width=1e-7, plateau=1e-7)
b = 0.5 * Drag(width=1e-7)
c = 0.8 * Cosine(width=2e-7, plateau=2e-7)
c1 = Carrier(phases=0, frequencies=2e7)
c2 = Carrier(phases=180, frequencies=5e7)
c3 = Carrier(phases=(0,0), frequencies=(8e6,9e6))
s = Sequence()
s.trigger_pos = 2e-6
s.register(3e-6, a, c1)
s.register(1e-6, b, c2)
s.register(2e-6, c, c3)
s.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …